In [1]:
from selenium import webdriver
from selenium.common.exceptions import WebDriverException as WDE
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST chromedriver version for 114.0.5735 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\0226p\.wdm\drivers\chromedriver\win32\114.0.5735.90]


In [3]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [4]:
# 검색 키워드"
SEARCH_KEYWORD = "자기소개서"

In [5]:
chrom_driver = 'D:\PJB\MyDatalab\Jupyter Notebook\chromedriver.exe'
driver = webdriver.Chrome(chrom_driver)
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(2)
# 무한 스크롤 함수 실행
scroll()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4880\2776537721.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrom_driver)


스크롤 완료


In [6]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'lxml')

In [7]:
# 첫 번째 컨텐츠의 제목, 채널명, url 정보 가져오기
# 첫 번째 컨텐츠 관련 부분 html 구조 떼어내기
# find: 해당 정보의 첫 번째 요소만 가져오기
test = soup.find("ytd-video-renderer", attrs={"class":'style-scope ytd-item-section-renderer'})

# 필요한 정보 가져오기
title = test.find("yt-formatted-string", attrs={"class":'style-scope ytd-video-renderer'}).get_text()
name = test.find("a", attrs={"class":'yt-simple-endpoint style-scope yt-formatted-string'}).get_text()
content_url = test.find("a", attrs={"class":'yt-simple-endpoint style-scope ytd-video-renderer'})["href"]

print(title, name, content_url)

면접관이 5초만 봐도 합격시키고 싶은 자소서 작성법 3가지 면접왕 이형 /watch?v=eGv5EAMF8OQ&pp=ygUP7J6Q6riw7IaM6rCc7ISc


In [8]:
# finish line까지 모든 검색 결과 정보 가져오기
# 모든 컨텐츠 관련 부분을 떼어내기
# find_all: 해당 정보의 모든 부분 가져오기
elem = soup.find_all("ytd-video-renderer", attrs={"class":'style-scope ytd-item-section-renderer'})

# 필요한 정보 가져오기
df = []
for t in elem:
    title = t.find("yt-formatted-string", attrs={"class":'style-scope ytd-video-renderer'}).get_text()
    name = t.find("a", attrs={"class":'yt-simple-endpoint style-scope yt-formatted-string'}).get_text()
    content_url = t.find("a", attrs={"class":'yt-simple-endpoint style-scope ytd-video-renderer'})["href"]
    df.append([title, 'https://www.youtube.com/'+content_url, name])

In [9]:
df

[['면접관이 5초만 봐도 합격시키고 싶은 자소서 작성법 3가지',
  'https://www.youtube.com//watch?v=eGv5EAMF8OQ&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  '면접왕 이형'],
 ['자소서 필수 문항',
  'https://www.youtube.com//watch?v=fudTkkL86IY&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  '면접왕 이형'],
 ['모든 합격 자소서 첫줄의 공통점 & 최고의 경험으로도 탈락하는 실수 feat. 서울특별시 [잡담캠프 하이라이트]',
  'https://www.youtube.com//watch?v=t2qvkL-pjtI&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  'AND (ft.인싸담당자)'],
 ['자소서 쓸때 듣는 음악ㅣ딱 2시간 집중해서 글쓰기ㅣ가사 없는 음악 playlist',
  'https://www.youtube.com//watch?v=ov0Mbm6FHu8&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  '전설의 글쓰기'],
 ['자기소개서 아무리 열심히 써도 이 5가지 표현을 안 바꾸면 탈락입니다｜해커스잡 조은희｜자소서, 자기소개서 예시',
  'https://www.youtube.com//watch?v=vopHqWi250o&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  '취업강의 1위 l 해커스잡·공기업'],
 ['면접관이 듣기만해도 불합격시키고 싶은 1분 자기소개',
  'https://www.youtube.com//watch?v=rZX2GsjLcww&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  '면접왕 이형'],
 ['자소서 지원동기 6단계만 따라 쓰면 합격합니다 [마스터 자소서 I EP 04. 지원동기]',
  'https://www.youtube.com//watch?v=OIliv2jpDyM&pp=ygUP7J6Q6riw7IaM6rCc7ISc',
  'AND (ft.인싸담당자)'],
 ["🚫'탈

In [10]:
from pandas import DataFrame

## 자료 저장
# 데이터 프레임 만들기
new = pd.DataFrame(columns=['제목', 'URL' , '게시자'])

# 자료 집어넣기
for i in range(len(df)):
    new.loc[i] = df[i]

# 저장하기
# 현재 작업폴더 안의 data 폴더에 저장
new.to_csv("D:\PJB\MyDatalab\Develop-Crawling\Crawling_MyDatalab\Output-data\youtube_data_v.1.0.csv", index=False)

# 브라우저 닫기
driver.close()